In [2]:
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
from typing import Dict, List

import matplotlib
import json
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

def save_stats(stats):
    with open("stats.json", "w") as f:
        json.dump(stats, f, sort_keys=True, indent=4)

def load_stats() -> Dict:
    with open("stats.json", "r") as f:
        return json.load(f)

def print_stats(stats):
    print(json.dumps(stats, indent=4, sort_keys=True))

def highlight_nodes(graph: nx.Graph, nodes: List = []):
    reset_colors(graph)
    for node in nodes:
        graph.nodes[node]["viz"]["color"] = {'r': 255, 'g': 0, 'b': 0, 'a': 1}

def highlight_nodes_importance(graph: nx.Graph, node_importance : Dict = {}):
    reset_colors(graph)
    min = np.min(list(node_importance.values()))
    max = np.max(list(node_importance.values()))
    norm = matplotlib.colors.Normalize(vmin=min, vmax=max)
    for node, importance in node_importance.items():
        g = int(255 * norm(importance))
        r = 255 - g
        graph.nodes[node]["viz"]["color"] = {'r': r, 'g': g, 'b': 0, 'a': 1}

def highlight_nodes_communities(graph: nx.Graph, communities: List[List] = []):
    reset_colors(graph)
    cmap: LinearSegmentedColormap = plt.get_cmap('hsv')
    for i, community in enumerate(communities):
        (r, g, b, a) = cmap(i / len(communities))
        r, g, b, a = int(r * 255), int(g * 255), int(b * 255), a
        for node in community:
            graph.nodes[node]["viz"]["color"] = {'r': r, 'g': g, 'b': b, 'a': a}


def reset_colors(graph: nx.Graph):
    for node in graph:
        if(not "viz" in graph.nodes[node]):
            graph.nodes[node]["viz"] = {}
        graph.nodes[node]["viz"]["color"] = {'r': 173, 'g': 216, 'b': 230, 'a': 1}


def circular_layout(graph: nx.Graph):
    pos = nx.circular_layout(graph, scale=4*graph.number_of_nodes())
    for node, position in pos.items():
        if(not "viz" in graph.nodes[node]):
            graph.nodes[node]["viz"] = {}
        graph.nodes[node]["viz"]["position"] = {"x": position[0], "y": position[1], "z": 0.0}


In [ ]:
USE_TEST_DATASET = False
IGNORE_PREVIOUS_STATS = False

if(not USE_TEST_DATASET):
    DATASET_PATH = "./datasets/facebook_combined.txt"

In [ ]:
if(USE_TEST_DATASET):
    G = nx.karate_club_graph()
else:
    G : nx.Graph = nx.read_edgelist(path=DATASET_PATH, create_using=nx.Graph, nodetype=int)

In [ ]:
if (IGNORE_PREVIOUS_STATS):
    STATS = {}
else:
    STATS = load_stats()

In [254]:
circular_layout(G)
reset_colors(G)

In [5]:
nx.write_gexf(G, "graphs/graph.gexf")

In [6]:
if(not "number_of_nodes" in STATS):
    STATS["number_of_nodes"] =  G.number_of_nodes()
    save_stats(STATS)

if(not "number_of_edges" in STATS):
    STATS["number_of_edges"] =  G.number_of_edges()
    save_stats(STATS)

if(not "diameter" in STATS):
    STATS["diameter"] = nx.diameter(G)
    save_stats(STATS)
    
if(not "radius" in STATS):
    STATS["radius"] = nx.radius(G)
    save_stats(STATS)


In [7]:
if(not "centers" in STATS):
    STATS["centers"] = nx.center(G)
    save_stats(STATS)

In [8]:
highlight_nodes(G, STATS["centers"])
nx.write_gexf(G, "graphs/centers.gexf")

In [9]:
if(not "degree_centrality" in STATS):
    STATS["degree_centrality"] = nx.degree_centrality(G)
    save_stats(STATS)

In [10]:
highlight_nodes_importance(G, STATS["degree_centrality"])
nx.write_gexf(G, "graphs/degree_centrality.gexf")

In [ ]:
# neighborhood_connectivity = nx.neighbor_connectivity(G)

In [11]:
if(not "eigenvector_centrality" in STATS):
    STATS["eigenvector_centrality"] = nx.eigenvector_centrality(G)
    save_stats(STATS)

In [12]:
highlight_nodes_importance(G, STATS["eigenvector_centrality"])
nx.write_gexf(G, "graphs/eigenvector_centrality.gexf")

In [263]:
if(not "pagerank" in STATS):
    STATS["pagerank"] = nx.pagerank(G)
    save_stats(STATS)

In [264]:
highlight_nodes_importance(G, STATS["pagerank"])
nx.write_gexf(G, "graphs/pagerank.gexf")

In [13]:
if(not "clustering_coeff" in STATS):
    STATS["clustering_coeff"] = nx.clustering(G)
    save_stats(STATS)

In [14]:
highlight_nodes_importance(G, STATS["clustering_coeff"])
nx.write_gexf(G, "graphs/clustering_coeff.gexf")

In [15]:
if(not "closeness_centrality" in STATS):
    STATS["closeness_centrality"] = nx.closeness_centrality(G)
    save_stats(STATS)

In [16]:
highlight_nodes_importance(G, STATS["closeness_centrality"])
nx.write_gexf(G, "graphs/closeness_centrality.gexf")

In [17]:
if(not "betweenness_centrality" in STATS):
    STATS["betweenness_centrality"] = nx.betweenness_centrality(G)
    save_stats(STATS)

In [18]:
highlight_nodes_importance(G, STATS["betweenness_centrality"])
nx.write_gexf(G, "graphs/betweenness_centrality.gexf")

In [19]:
if(not "k_clique_communities" in STATS):
    STATS["k_clique_communities"] = [list(x) for x in nx.community.k_clique_communities(G, 4)]
    save_stats(STATS)

In [ ]:
highlight_nodes_communities(G, STATS["k_clique_communities"])
nx.write_gexf(G, "graphs/k_clique_communities.gexf")

In [ ]:
if(not "louvain_communities" in STATS):
    STATS["louvain_communities"] = [list(x) for x in nx.community.louvain_communities(G)]
    save_stats(STATS)

In [ ]:
highlight_nodes_communities(G, STATS["louvain_communities"])
nx.write_gexf(G, "graphs/louvain_communities.gexf")

In [ ]:
if(not "girvan_newman" in STATS):
    communities = list(nx.community.girvan_newman(G))
    for index, tup in enumerate(communities):
        communities[index] = list(tup)
    for tup in communities:
        for index, sett in enumerate(tup):
            tup[index] = list(sett)
    STATS["girvan_newman"] = communities
    save_stats(STATS)

In [ ]:
for index, community in enumerate(communities, 1):
    highlight_nodes_communities(G, community)
    nx.write_gexf(G, f"graphs/girvan_newman/girvan_newman_{index}.gexf")